In [1]:
!pip install nltk



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%time
import os
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
import numpy as np
import pandas as pd
import sklearn

import string
import re
import nltk

CPU times: total: 0 ns
Wall time: 0 ns


In [4]:
%time
# read the csv file
train_df = pd.read_csv("output.csv")
display(train_df.shape, train_df.head())

CPU times: total: 0 ns
Wall time: 0 ns


(50, 17)

,location,contract_time,redirect_url,latitude,description,__CLASS__,id,salary_min,category,created,salary_max,longitude,salary_is_predicted,title,adref,company,contract_type
0,{'__CLASS__': 'Adzuna::API::Response::Location...,full_time,https://www.adzuna.com/land/ad/4456537799?se=X...,39.132050,Build Something to Be Proud Of. Captivation So...,Adzuna::API::Response::Job,4456537799,145812.33,{'__CLASS__': 'Adzuna::API::Response::Category...,2023-11-29T12:57:48Z,145812.33,-76.799422,1,Software Engineer (Mid/Senior) - Java/Python/S...,eyJhbGciOiJIUzI1NiJ9.eyJzIjoiWENhaHNkNlM3aEdOV...,"{'display_name': 'Captivation Software', '__CL...",NaN
1,{'__CLASS__': 'Adzuna::API::Response::Location...,full_time,https://www.adzuna.com/land/ad/4456528057?se=X...,39.132050,Build Something to Be Proud Of. Captivation So...,Adzuna::API::Response::Job,4456528057,197734.62,"{'label': 'IT Jobs', 'tag': 'it-jobs', '__CLAS...",2023-11-29T12:56:04Z,197734.62,-76.799422,1,Software Engineer 2 (Hybrid) - Python/Java/Dat...,eyJhbGciOiJIUzI1NiJ9.eyJzIjoiWENhaHNkNlM3aEdOV...,"{'display_name': 'Captivation Software', '__CL...",NaN
2,"{'display_name': 'Baltimore, Baltimore County'...",full_time,https://www.adzuna.com/land/ad/4463988439?se=X...,39.296337,"Center 1 (19052), United States of America, Mc...",Adzuna::API::Response::Job,4463988439,54100.19,"{'tag': 'it-jobs', 'label': 'IT Jobs', '__CLAS...",2023-12-04T11:56:39Z,54100.19,-76.621054,1,"Senior Software Engineer, Back End (Python, Java)",eyJhbGciOiJIUzI1NiJ9.eyJpIjoiNDQ2Mzk4ODQzOSIsI...,"{'display_name': 'Capital One', '__CLASS__': '...",contract
3,{'__CLASS__': 'Adzuna::API::Response::Location...,full_time,https://www.adzuna.com/land/ad/4463988504?se=X...,37.560000,"Center 1 (19052), United States of America, Mc...",Adzuna::API::Response::Job,4463988504,52994.28,{'__CLASS__': 'Adzuna::API::Response::Category...,2023-12-04T11:56:40Z,52994.28,-77.450000,1,"Senior Software Engineer, Back End (Python, Java)",eyJhbGciOiJIUzI1NiJ9.eyJzIjoiWENhaHNkNlM3aEdOV...,"{'display_name': 'Capital One', '__CLASS__': '...",contract
4,"{'area': ['US', 'Virginia', 'Harrisonburg City...",full_time,https://www.adzuna.com/land/ad/4463988520?se=X...,38.414210,"Center 1 (19052), United States of America, Mc...",Adzuna::API::Response::Job,4463988520,53714.87,"{'label': 'IT Jobs', 'tag': 'it-jobs', '__CLAS...",2023-12-04T11:56:40Z,53714.87,-78.892628,1,"Senior Software Engineer, Back End (Python, Java)",eyJhbGciOiJIUzI1NiJ9.eyJzIjoiWENhaHNkNlM3aEdOV...,{'__CLASS__': 'Adzuna::API::Response::Company'...,contract


In [5]:
# display(train_df[~train_df["location" and "Position" and "Company" and "details"].isnull()].head())
null_counts = train_df.isnull().sum()
print("Null values per column:")
print(null_counts)

null_percentages = (train_df.isnull().sum() / len(train_df)) * 100
print("\nPercentage of null values per column:")
print(null_percentages)

Null values per column:
location                0
contract_time           0
redirect_url            0
latitude                2
description             0
__CLASS__               0
id                      0
salary_min              0
category                0
created                 0
salary_max              0
longitude               2
salary_is_predicted     0
title                   0
adref                   0
company                 0
contract_type          14
dtype: int64

Percentage of null values per column:
location                0.0
contract_time           0.0
redirect_url            0.0
latitude                4.0
description             0.0
__CLASS__               0.0
id                      0.0
salary_min              0.0
category                0.0
created                 0.0
salary_max              0.0
longitude               4.0
salary_is_predicted     0.0
title                   0.0
adref                   0.0
company                 0.0
contract_type          28.0
dtype:

In [6]:
display(train_df[~train_df["location" and "title" and "company" and "description"].isnull()].head())


,location,contract_time,redirect_url,latitude,description,__CLASS__,id,salary_min,category,created,salary_max,longitude,salary_is_predicted,title,adref,company,contract_type
0,{'__CLASS__': 'Adzuna::API::Response::Location...,full_time,https://www.adzuna.com/land/ad/4456537799?se=X...,39.132050,Build Something to Be Proud Of. Captivation So...,Adzuna::API::Response::Job,4456537799,145812.33,{'__CLASS__': 'Adzuna::API::Response::Category...,2023-11-29T12:57:48Z,145812.33,-76.799422,1,Software Engineer (Mid/Senior) - Java/Python/S...,eyJhbGciOiJIUzI1NiJ9.eyJzIjoiWENhaHNkNlM3aEdOV...,"{'display_name': 'Captivation Software', '__CL...",NaN
1,{'__CLASS__': 'Adzuna::API::Response::Location...,full_time,https://www.adzuna.com/land/ad/4456528057?se=X...,39.132050,Build Something to Be Proud Of. Captivation So...,Adzuna::API::Response::Job,4456528057,197734.62,"{'label': 'IT Jobs', 'tag': 'it-jobs', '__CLAS...",2023-11-29T12:56:04Z,197734.62,-76.799422,1,Software Engineer 2 (Hybrid) - Python/Java/Dat...,eyJhbGciOiJIUzI1NiJ9.eyJzIjoiWENhaHNkNlM3aEdOV...,"{'display_name': 'Captivation Software', '__CL...",NaN
2,"{'display_name': 'Baltimore, Baltimore County'...",full_time,https://www.adzuna.com/land/ad/4463988439?se=X...,39.296337,"Center 1 (19052), United States of America, Mc...",Adzuna::API::Response::Job,4463988439,54100.19,"{'tag': 'it-jobs', 'label': 'IT Jobs', '__CLAS...",2023-12-04T11:56:39Z,54100.19,-76.621054,1,"Senior Software Engineer, Back End (Python, Java)",eyJhbGciOiJIUzI1NiJ9.eyJpIjoiNDQ2Mzk4ODQzOSIsI...,"{'display_name': 'Capital One', '__CLASS__': '...",contract
3,{'__CLASS__': 'Adzuna::API::Response::Location...,full_time,https://www.adzuna.com/land/ad/4463988504?se=X...,37.560000,"Center 1 (19052), United States of America, Mc...",Adzuna::API::Response::Job,4463988504,52994.28,{'__CLASS__': 'Adzuna::API::Response::Category...,2023-12-04T11:56:40Z,52994.28,-77.450000,1,"Senior Software Engineer, Back End (Python, Java)",eyJhbGciOiJIUzI1NiJ9.eyJzIjoiWENhaHNkNlM3aEdOV...,"{'display_name': 'Capital One', '__CLASS__': '...",contract
4,"{'area': ['US', 'Virginia', 'Harrisonburg City...",full_time,https://www.adzuna.com/land/ad/4463988520?se=X...,38.414210,"Center 1 (19052), United States of America, Mc...",Adzuna::API::Response::Job,4463988520,53714.87,"{'label': 'IT Jobs', 'tag': 'it-jobs', '__CLAS...",2023-12-04T11:56:40Z,53714.87,-78.892628,1,"Senior Software Engineer, Back End (Python, Java)",eyJhbGciOiJIUzI1NiJ9.eyJzIjoiWENhaHNkNlM3aEdOV...,{'__CLASS__': 'Adzuna::API::Response::Company'...,contract


In [7]:
train_df["description"] = train_df["description"].astype(str).apply(lambda x: x.lower())
display(train_df.head())

,location,contract_time,redirect_url,latitude,description,__CLASS__,id,salary_min,category,created,salary_max,longitude,salary_is_predicted,title,adref,company,contract_type
0,{'__CLASS__': 'Adzuna::API::Response::Location...,full_time,https://www.adzuna.com/land/ad/4456537799?se=X...,39.132050,build something to be proud of. captivation so...,Adzuna::API::Response::Job,4456537799,145812.33,{'__CLASS__': 'Adzuna::API::Response::Category...,2023-11-29T12:57:48Z,145812.33,-76.799422,1,Software Engineer (Mid/Senior) - Java/Python/S...,eyJhbGciOiJIUzI1NiJ9.eyJzIjoiWENhaHNkNlM3aEdOV...,"{'display_name': 'Captivation Software', '__CL...",NaN
1,{'__CLASS__': 'Adzuna::API::Response::Location...,full_time,https://www.adzuna.com/land/ad/4456528057?se=X...,39.132050,build something to be proud of. captivation so...,Adzuna::API::Response::Job,4456528057,197734.62,"{'label': 'IT Jobs', 'tag': 'it-jobs', '__CLAS...",2023-11-29T12:56:04Z,197734.62,-76.799422,1,Software Engineer 2 (Hybrid) - Python/Java/Dat...,eyJhbGciOiJIUzI1NiJ9.eyJzIjoiWENhaHNkNlM3aEdOV...,"{'display_name': 'Captivation Software', '__CL...",NaN
2,"{'display_name': 'Baltimore, Baltimore County'...",full_time,https://www.adzuna.com/land/ad/4463988439?se=X...,39.296337,"center 1 (19052), united states of america, mc...",Adzuna::API::Response::Job,4463988439,54100.19,"{'tag': 'it-jobs', 'label': 'IT Jobs', '__CLAS...",2023-12-04T11:56:39Z,54100.19,-76.621054,1,"Senior Software Engineer, Back End (Python, Java)",eyJhbGciOiJIUzI1NiJ9.eyJpIjoiNDQ2Mzk4ODQzOSIsI...,"{'display_name': 'Capital One', '__CLASS__': '...",contract
3,{'__CLASS__': 'Adzuna::API::Response::Location...,full_time,https://www.adzuna.com/land/ad/4463988504?se=X...,37.560000,"center 1 (19052), united states of america, mc...",Adzuna::API::Response::Job,4463988504,52994.28,{'__CLASS__': 'Adzuna::API::Response::Category...,2023-12-04T11:56:40Z,52994.28,-77.450000,1,"Senior Software Engineer, Back End (Python, Java)",eyJhbGciOiJIUzI1NiJ9.eyJzIjoiWENhaHNkNlM3aEdOV...,"{'display_name': 'Capital One', '__CLASS__': '...",contract
4,"{'area': ['US', 'Virginia', 'Harrisonburg City...",full_time,https://www.adzuna.com/land/ad/4463988520?se=X...,38.414210,"center 1 (19052), united states of america, mc...",Adzuna::API::Response::Job,4463988520,53714.87,"{'label': 'IT Jobs', 'tag': 'it-jobs', '__CLAS...",2023-12-04T11:56:40Z,53714.87,-78.892628,1,"Senior Software Engineer, Back End (Python, Java)",eyJhbGciOiJIUzI1NiJ9.eyJzIjoiWENhaHNkNlM3aEdOV...,{'__CLASS__': 'Adzuna::API::Response::Company'...,contract


In [9]:
train_df.shape

(50, 17)

In [12]:
import PyPDF2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(pdf_reader.pages)):
            text += pdf_reader.pages[page_num].extract_text()
    return text

resume_text = extract_text_from_pdf('JinayPanchalResumeH.pdf')


train_df["combined_text"] = train_df["location"] + ' ' + train_df["title"] + ' ' + train_df["company"] + ' ' + train_df["description"]
train_df["combined_text"] = train_df["combined_text"].fillna('')

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(train_df["combined_text"])

resume_tfidf_vector = tfidf_vectorizer.transform([resume_text])

cosine_similarities = linear_kernel(resume_tfidf_vector, tfidf_matrix).flatten()
train_df["cosine_similarity"] = cosine_similarities

top_jobs = train_df.nlargest(10, "cosine_similarity")[["location", "title", "company", "description", "cosine_similarity"]]
display(top_jobs)


,location,title,company,description,cosine_similarity
0,{'__CLASS__': 'Adzuna::API::Response::Location...,Software Engineer (Mid/Senior) - Java/Python/S...,"{'display_name': 'Captivation Software', '__CL...",build something to be proud of. captivation so...,0.180681
28,"{'display_name': 'Savage, Anne Arundel County'...",Cloud Software Engineer 2 - Java/MySQL/Map Red...,{'__CLASS__': 'Adzuna::API::Response::Company'...,build something to be proud of. captivation so...,0.158864
21,"{'display_name': 'Valparaiso, Porter County', ...","Senior Software Engineer, Full Stack (Java, Py...","{'display_name': 'Capital One', '__CLASS__': '...","center 1 (19052), united states of america, mc...",0.128831
25,"{'display_name': 'Savage, Anne Arundel County'...",Software Engineer 2 - Linux/Bash/Python/Java/S...,"{'display_name': 'Captivation Software', '__CL...",build something to be proud of. captivation so...,0.124750
26,{'__CLASS__': 'Adzuna::API::Response::Location...,"Lead Software Engineer, Full Stack (Java, Pyth...","{'display_name': 'Capital One', '__CLASS__': '...","center 1 (19052), united states of america, mc...",0.116277
48,{'__CLASS__': 'Adzuna::API::Response::Location...,"Senior Software Engineer, Full Stack (Java/AWS)","{'display_name': 'Jobs for Humanity', '__CLASS...",job description position: senior software engi...,0.115191
1,{'__CLASS__': 'Adzuna::API::Response::Location...,Software Engineer 2 (Hybrid) - Python/Java/Dat...,"{'display_name': 'Captivation Software', '__CL...",build something to be proud of. captivation so...,0.111375
13,"{'display_name': 'South Bend, St. Joseph Count...","Senior Software Engineer, Full Stack (Java, Py...","{'display_name': 'Capital One', '__CLASS__': '...","center 1 (19052), united states of america, mc...",0.102952
7,"{'display_name': 'Fredericksburg City, Virgini...","Lead Software Engineer, Full Stack (Java, Python)","{'display_name': 'Capital One', '__CLASS__': '...","center 1 (19052), united states of america, mc...",0.100810
8,"{'display_name': 'Charlottesville City, Virgin...","Lead Software Engineer, Full Stack (Java, Python)",{'__CLASS__': 'Adzuna::API::Response::Company'...,"center 1 (19052), united states of america, mc...",0.100810


In [13]:
import pandas as pd
import PyPDF2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

def read_job_dataset(file_path):
    return pd.read_csv(file_path)

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(pdf_reader.pages)):
            text += pdf_reader.pages[page_num].extract_text()
    return text

def preprocess_job_dataset(df):
    df["description"] = df["description"].astype(str).apply(lambda x: x.lower())
    df["combined_text"] = df["location"] + ' ' + df["title"] + ' ' + df["company"] + ' ' + df["description"]
    df["combined_text"] = df["combined_text"].fillna('')
    return df

def calculate_cosine_similarity(train_df, resume_text):
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(train_df["combined_text"])
    
    resume_tfidf_vector = tfidf_vectorizer.transform([resume_text])
    
    cosine_similarities = linear_kernel(resume_tfidf_vector, tfidf_matrix).flatten()
    train_df["cosine_similarity"] = cosine_similarities
    return train_df

def get_top_matching_jobs(train_df, n=10):
    return train_df.nlargest(n, "cosine_similarity")[["location", "title", "company", "description", "cosine_similarity"]]

# Example usage
train_df = read_job_dataset("output.csv")
train_df = preprocess_job_dataset(train_df)

resume_text = extract_text_from_pdf('JinayPanchalResumeH.pdf')

train_df = calculate_cosine_similarity(train_df, resume_text)

top_jobs = get_top_matching_jobs(train_df)
display(top_jobs)


,location,title,company,description,cosine_similarity
0,{'__CLASS__': 'Adzuna::API::Response::Location...,Software Engineer (Mid/Senior) - Java/Python/S...,"{'display_name': 'Captivation Software', '__CL...",build something to be proud of. captivation so...,0.180681
28,"{'display_name': 'Savage, Anne Arundel County'...",Cloud Software Engineer 2 - Java/MySQL/Map Red...,{'__CLASS__': 'Adzuna::API::Response::Company'...,build something to be proud of. captivation so...,0.158864
21,"{'display_name': 'Valparaiso, Porter County', ...","Senior Software Engineer, Full Stack (Java, Py...","{'display_name': 'Capital One', '__CLASS__': '...","center 1 (19052), united states of america, mc...",0.128831
25,"{'display_name': 'Savage, Anne Arundel County'...",Software Engineer 2 - Linux/Bash/Python/Java/S...,"{'display_name': 'Captivation Software', '__CL...",build something to be proud of. captivation so...,0.124750
26,{'__CLASS__': 'Adzuna::API::Response::Location...,"Lead Software Engineer, Full Stack (Java, Pyth...","{'display_name': 'Capital One', '__CLASS__': '...","center 1 (19052), united states of america, mc...",0.116277
48,{'__CLASS__': 'Adzuna::API::Response::Location...,"Senior Software Engineer, Full Stack (Java/AWS)","{'display_name': 'Jobs for Humanity', '__CLASS...",job description position: senior software engi...,0.115191
1,{'__CLASS__': 'Adzuna::API::Response::Location...,Software Engineer 2 (Hybrid) - Python/Java/Dat...,"{'display_name': 'Captivation Software', '__CL...",build something to be proud of. captivation so...,0.111375
13,"{'display_name': 'South Bend, St. Joseph Count...","Senior Software Engineer, Full Stack (Java, Py...","{'display_name': 'Capital One', '__CLASS__': '...","center 1 (19052), united states of america, mc...",0.102952
7,"{'display_name': 'Fredericksburg City, Virgini...","Lead Software Engineer, Full Stack (Java, Python)","{'display_name': 'Capital One', '__CLASS__': '...","center 1 (19052), united states of america, mc...",0.100810
8,"{'display_name': 'Charlottesville City, Virgin...","Lead Software Engineer, Full Stack (Java, Python)",{'__CLASS__': 'Adzuna::API::Response::Company'...,"center 1 (19052), united states of america, mc...",0.100810
